# 1 

In [1]:
using JuMP, Clp, NamedArrays

num_orders = 160 # total orders 
w = [:M, :P, :S, :N] # warehouses 
dc = [:A, :L, :B] # distribution centers 
modes = [:air, :rail] # possible modes of transportation 
max_w_cap = Dict(zip(w,[45,50,70,30])) # capacity of each warehouse 
min_air_cap = Dict(zip(dc,[5,10,15]))
nodes = [] # fill in nodes 
arcs = []; 
# fill in arcs. you might want to use tuples 
# e.g., arc from node 1 to node2: (:node1,:node2)

raw_air = [12 11 100000
        12 14 20
        100000 100000 18
        9 8 100000]

raw_rail = [100000 100000 100000
        10 12 100000
        100000 100000 100000
        7 100000 5 ]

air = NamedArray(raw_air, (w,dc),("w","dc"))
rail = NamedArray(raw_rail, (w,dc), ("w","dc"))

m = Model(solver=ClpSolver())
@variable(m, 0<=x[w, dc] <=45)
@variable(m, 0<=y[w, dc] <=60)

@objective(m, Min, sum(x[i,j]*air[i,j]+y[i,j]*rail[i,j] for i in w, j in dc))

for i in w
    @constraint(m, sum(x[i,j]+y[i,j] for j in dc)<=max_w_cap[i])
end

@constraint(m, sum(x) + sum(y) == 160)

for j in dc
    @constraint(m, sum(x[i,j] for i in w) >= min_air_cap[j])
end

solve(m)

println("Cost: \$", getobjectivevalue(m),"k")

println(getvalue(x))

#assignment1 = NamedArray( [ (getvalue(x[i,j])) for i in w, j in dc ], (w, dc), ("w","dc"))
assignment2 = NamedArray( [ (getvalue(y[i,j])) for i in w, j in dc ], (w, dc), ("w","dc"))


Cost: $1780.0k
x: 2 dimensions:
[M,:]
  [M,A] = 5.0
  [M,L] = 40.0
  [M,B] = 0.0
[P,:]
  [P,A] = 0.0
  [P,L] = 0.0
  [P,B] = 0.0
[S,:]
  [S,A] = 0.0
  [S,L] = 0.0
  [S,B] = 35.0
[N,:]
  [N,A] = 0.0
  [N,L] = 0.0
  [N,B] = 0.0


4×3 Named Array{Float64,2}
w ╲ dc │   :A    :L    :B
───────┼─────────────────
:M     │  0.0   0.0   0.0
:P     │ 50.0   0.0   0.0
:S     │  0.0   0.0   0.0
:N     │  0.0   0.0  30.0

# 2

In [2]:
using JuMP, Clp, NamedArrays

node = [:1, :2, :3, :4, :5, :6, :7, :8, :9, :10]

cap_raw = [
0 0 21 11 16 0 0 0 0 0 
0 0 0 22 5 0 0 0 0 0 
0 0 0 0 9 0 16 0 0 0 
0 0 0 0 2 23 0 0 0 0
0 0 0 0 0 0 0 14 10 16 
0 0 0 0 0 0 0 0 10 9 
0 0 0 0 0 0 0 8 10 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
]
cap = NamedArray(cap_raw, (node,node), ("node1","node2"))

source = Dict(zip(node,[30,27,0,0,0,0,0,0,0,0]))
sink = Dict(zip(node,[0,0,0,0,0,0,0,17,15,21]))

m = Model(solver = ClpSolver())

@variable(m, x[node,node] >=0)

for i in node
    for j in node
        @constraint(m, 0<= x[i,j] <=cap[i,j])
    end
end

for i = 3:7
    @constraint(m, sum(x[j,i] for j in node) == sum(x[i,k] for k in node))
end

for i = 1:2
    @constraint(m, sum(x[i,j] for j in node) <= source[i])
end

for i = 8:10
    @constraint(m, sum(x[j,i] for j in node) >= sink[i])
end
#@constraint(m, a[i = 1:10, j=1:10], x[i,j] <= [i,j])

@objective(m, Max, sum(x[i,8]+x[i,9]+x[i,10] for i in node))
solve(m)

println("Max flow: ", getobjectivevalue(m)) # need to take negative

assignment = NamedArray( [ (getvalue(x[i,j])) for i in node, j in node ], (node, node), ("f","t"))


Max flow: 56.0


10×10 Named Array{Float64,2}
f ╲ t │    1     2     3     4     5     6     7     8     9    10
──────┼───────────────────────────────────────────────────────────
1     │  0.0   0.0  16.0   0.0  14.0   0.0   0.0   0.0   0.0   0.0
2     │  0.0   0.0   0.0  21.0   5.0   0.0   0.0   0.0   0.0   0.0
3     │  0.0   0.0   0.0   0.0   0.0   0.0  16.0   0.0   0.0   0.0
4     │  0.0   0.0   0.0   0.0   2.0  19.0   0.0   0.0   0.0   0.0
5     │  0.0   0.0   0.0   0.0   0.0   0.0   0.0   9.0   0.0  12.0
6     │  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  10.0   9.0
7     │  0.0   0.0   0.0   0.0   0.0   0.0   0.0   8.0   8.0   0.0
8     │  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
9     │  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
10    │  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0

Method 1
Yes, it can
This one is the max flow fulfill the citis' water demand.

In [2]:
using JuMP, Clp

# incidence matrix
#s1,s2||13,14,15,24,25,||35,37,45,46,||58,59,510,69,610,78,79||8s,9s,10s
A = [
1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
-1 0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
0 -1  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0 -1  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0 
0  0  0 -1  0 -1  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0 
0  0  0  0 -1  0 -1 -1  0 -1  0  1  1  1  0  0  0  0  0  0  0 
0  0  0  0  0  0  0  0  0  0 -1  0  0  0  1  1  0  0  0  0  0 
0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  1  1  0  0  0 
0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0 -1  0  1  0  0 
0  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0 -1  0  1  0 
0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  1 
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1    
    ] 

# dummy arc from sink (5) to source (0)
A = [A [-1;0;0;0;0;0;0;0;0;0;0;1]]

# supply and demand
b = [0, 0, 0, 0, 0, 0,0,0,0,0,0,0]

# costs
c = [0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0, 0,0,0,-1]

# capacities on each arc. make dummy arc capacity "big enough"
cap = [30,27,21,11,16,22,5,9,16,2,23,14,10,16,10,9,8,10,100,100,100,100]

m = Model(solver=ClpSolver())

@variable(m, x[1:22] >= 0)

@constraint(m, A*x .== b)

@constraint(m, x .<= cap)

@objective(m, Min, dot(c,x))

@constraint(m, x[19] >= 17)
@constraint(m, x[20] >=15)
@constraint(m, x[21] >=21)

solve(m)

println("Max flow: ", -getobjectivevalue(m)) # need to take negative

println("Flow on each arc: ")
for i = 1:22
    print(i, ": ")
    println(getvalue(x[i]))
end


Max flow: 56.0
Flow on each arc: 
1: 30.0
2: 26.0
3: 14.0
4: 0.0
5: 16.0
6: 21.0
7: 5.0
8: 9.0
9: 5.0
10: 2.0
11: 19.0
12: 14.0
13: 6.0
14: 12.0
15: 10.0
16: 9.0
17: 5.0
18: 0.0
19: 19.0
20: 16.0
21: 21.0
22: 56.0


In [ ]:
Method 2
Yes, it can
This one is the max flow fulfill the citis' water demand.

# 3

Math Model:

We extend the restrains on sinks to all, we creat a new vector to record the lower bound (d).(easier to write in code)

Primal:
Max: -cx

x>=0
x<=cap
x>=d (restraint on these sinks)
A*x == b (b is all zero)

Dual:
(t(22) is the parameter for arch, s(12) is the parameter for node, v for the restrain on sinks)
Min: cap'*t + b'* s + d'*v(since b is all zero it can be reduced to cap'*t+ d'*v)

s free
t >= 0
v <= 0
t[i] + (A'*s)[i] +v[i]>= -c[i]


In [53]:
using JuMP, Clp

m = Model(solver=ClpSolver())

cap = [30,27,21,11,16,22,5,9,16,2,23,14,10,16,10,9,8,10,100,100,100,100]

A = [
1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
-1 0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
0 -1  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0 -1  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0 
0  0  0 -1  0 -1  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0 
0  0  0  0 -1  0 -1 -1  0 -1  0  1  1  1  0  0  0  0  0  0  0 
0  0  0  0  0  0  0  0  0  0 -1  0  0  0  1  1  0  0  0  0  0 
0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  1  1  0  0  0 
0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0 -1  0  1  0  0 
0  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0 -1  0  1  0 
0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  1 
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1    
    ] 

# dummy arc from sink (5) to source (0)
A = [A [-1;0;0;0;0;0;0;0;0;0;0;1]]

# supply and demand
b = [0, 0, 0, 0, 0, 0,0,0,0,0,0,0]

# costs
c = [0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0, 0,0,0,-1]

# bounds for arch 19,20,21
d = [0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,17,15,21,0]

@variable(m, s[1:12])
@variable(m, t[1:22] >= 0)
@variable(m, v[1:22] <= 0)

@objective(m, Min, sum(cap'[i] * t[i] + d[i]'*v[i] for i =1:22))

for i = 1:22
    @constraint(m, t[i] + (A'*s)[i] +v[i]>= -c[i])
end



solve(m)

println("Max flow: ", getobjectivevalue(m)) # need to take negative
println(getvalue(s))
println(getvalue(t))
println(getvalue(v))


Max flow: 56.0
[-1.0, -0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Since the optimal solution exists.
By theorem:
Therefore, the dual solution is equal to the primal solution